# Grounded Rewards

In [1]:
import ipywidgets as widgets
from traitlets import Dict, Unicode, validate

class Display(widgets.DOMWidget):
    _view_name = Unicode('Display').tag(sync=True)
    _view_module = Unicode('display').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)
    state = Dict({"values": []}).tag(sync=True)

In [2]:
%%javascript
// Reset the loader's internal state to forget about the previous definition of the module
require.undef('display');

require.config({
  //Define 3rd party plugins dependencies
  paths: {
    fabric: "https://cdnjs.cloudflare.com/ajax/libs/fabric.js/2.7.0/fabric.min"
  }
});

define('display', ["@jupyter-widgets/base", "fabric"], function(widgets, fabric) {

    var Display = widgets.DOMWidgetView.extend({

        render: function() {
            const canvas = document.createElement('canvas');
            canvas.id = 'canvas';
            canvas.width = 1000;
            canvas.height = 500;
            var ctx = canvas.getContext("2d");
            ctx.fillStyle = "blue";
            ctx.fillRect(0, 0, canvas.width, canvas.height);
            this.el.appendChild(canvas);

            const fabricCanvas = new fabric.Canvas(canvas);

            // Create a starting rect (useful to see something is working)
            const shape = new fabric.Circle({
                top : 100,
                left : 100,
                radius : 20,
                fill : '#5BC8F7'
            });

            fabricCanvas.add(shape);
            
            // Create a list of objects to re-use
            const shapes = [shape];
            
            // Set up our listener
            const onStateChanged = this.handleStateChanged.bind(this, fabricCanvas, shapes);
            this.model.on('change:state', onStateChanged);
        },
        
        handleStateChanged: function(fabricCanvas, shapes) {
            const vals = this.model.get('state').values;
            
            for (let i=0; i < vals.length; i++) {
                let shape;
                // Create a new shape if our list of vals has increased in length
                if (shapes[i] === undefined) {
                    shape = new fabric.Circle({
                        top : 100,
                        left : 100,
                        radius : 20,
                        fill : '#F6C7BE'
                    });
                    fabricCanvas.add(shape);
                    shapes.push(shape);
                } else {
                    shape = shapes[i];
                }
                // Update the shape to the correct location
                const top = vals[i];
                const left = 500 - (i * (shape.radius * 3));
                shape.set({
                    top,
                    left
                });
                
            }
            
            // Render all rects;
            fabricCanvas.renderAll();
        }
    });

    return {
        Display
    };
});

<IPython.core.display.Javascript object>

In [3]:
# Instantiate the Python widget.
display = Display()

In [4]:
# Invoke the __repr__() of the widget which actually causes the JavaScript widget to be drawn.
display

Display(state={'values': []})

In [5]:
# Add some new values to our state and watch our display update live!
import math
from time import sleep
from collections import deque

i = 0
q = deque(maxlen=9)
while True:
    new_state = display.state.copy()
    new_val = (math.sin(i) + 1) * 200
    q.appendleft(new_val)
    new_state['values'] = list(q)
    display.state = new_state
    i += 0.05
    sleep(0.02)

KeyboardInterrupt: 

# Full Puzzle

https://miro.com/app/board/o9J_kwPmu2M=/

Set Point -> Value -> Error -> Perception -> Controller -> Feedback -> Control Element Action

observation ->


# Evolve a controller

We want to use either an Evolution Strategies (ES) or Genetic Algorithm (GA) approach to create a controller for a continuous dynamic process.

# Citations

## Paper Links

"Another distinguishing feature of drives are their homeostatic nature. For animals to survive, they must maintain a variety of critical parameters (such as temperature, energy level, amount of fluids, etc.) within a bounded range. As such, the drives keep changing in intensity to reflect the ongoing needs of the robot and the urgency for tending to them.  There is a desired operational point for each drive and an acceptable bounds of operation around that point.  We call this range the homeostatic regime. As long as a drive is within the homeostatic regime, the robot’s “needs” are being adequately met."

http://robotic.media.mit.edu/wp-content/uploads/sites/7/2015/01/Breazeal-AAAI-98.pdf


Figure 1

https://www.researchgate.net/profile/Oscar_Deniz/publication/228709943_A_Proposal_of_a_Homeostatic_Regulation_Mechanism_for_a_Vision_System/links/5419ded50cf25ebee98882c9/A-Proposal-of-a-Homeostatic-Regulation-Mechanism-for-a-Vision-System.pdf

Figure 2

https://ieeexplore.ieee.org/document/8408296

## Web Links (Need to Find Papers)

"To survive, animals must maintain certain critical parameters within a bounded range. For instance, an animal must regulate its temperature, energy level, amount of fluids, etc. Maintaining each critical parameter requires that the animal come into contact with the corresponding satiatory stimulus (shelter, food, water, etc.) at the right time. The process by which these critical parameters are maintained is generally referred to as homeostatic regulation.

In a simplified view, each satiatory stimulus can be thought of as an innately specified need. In broad terms, there is a desired fixed point of operation for each parameter, and an allowable bounds of operation around that point (see figure). As the critical parameter moves away from the desired point of operation, the animal becomes more strongly motivated to behave in ways that will restore that parameter. The physiological mechanisms that serve to regulate these needs, driving the animal into contact with the needed stimulus at the appropriate time, are quite complex and distinct."

http://www.ai.mit.edu/projects/sociable/homeostatic-regulation.html